In [1]:
import os
import json
import copy
import numpy as np
import pandas as pd
from pathlib import Path
import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatools.analyzer import *

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [10]:
def load_convs(path, filename):
    with open(path+filename, "r") as f:
        json_data = json.load(f)
        # length = json_data["length"]
        convs = json_data["convs"]
    return convs

In [11]:
cont_dir = "continue/"
cont_path = "../../corpus/nucc/" + cont_dir
length = 2
filename = "cont{0}.json".format(length)

In [12]:
convs = load_convs(cont_path, filename)

In [13]:
X_str = [conv for data in convs for conv in data]
X_utt = [utt for data in convs for conv in data for utt in conv]

In [16]:
import random
import copy
def fake_error(convs):
    utt_list = [conv[-1] for conv in convs]
    # print(utt_list)
    new_conv = copy.deepcopy(convs)
    for conv in new_conv:
        conv[-1] = random.choice(utt_list)
    return new_conv


In [17]:
UT = fake_error( X_str[:int( len(X_str)*0.2) ] ) 
UT_NO = X_str[int( len(X_str)*0.2):]
print("UT: {0}".format(len(UT)))
print("UT_NO: {0}".format(len(UT_NO)))

UT: 2416
UT_NO: 9667


In [18]:
from pyknp import Juman
from sentence_transformers import SentenceTransformer
import scipy.spatial
model_path = "/home/yamada/Downloads/training_bert_japanese"
Nmodel = SentenceTransformer(model_path, show_progress_bar=False)
emb_dim = Nmodel.encode(["お辞儀をしている男性会社員"])[0].shape[0]

In [24]:
def make_X(convs, max_len):
    # emb_dim = nlp("形態素").vector.shape
    X_data = []
    
    for conv in convs :
        sentence_vectors = Nmodel.encode(conv)
        X_data.append(sentence_vectors[1]- sentence_vectors[0])
    return np.array(X_data)

In [27]:
X = make_X(UT + UT_NO, length)
y = np.concatenate( [ np.ones(len(UT)), np.zeros(len(UT_NO)) ] )

In [28]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)


In [29]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier()
clf.fit(X_train, y_train)

AdaBoostClassifier()

In [31]:
y_pred = clf.predict(X_test)

In [32]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))
print('precision = ', precision_score(y_true=y_test, y_pred=y_pred))
print('recall = ', recall_score(y_true=y_test, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y_test, y_pred=y_pred))

confusion matrix = 
 [[2727  150]
 [ 661   87]]
accuracy =  0.7762758620689655
precision =  0.3670886075949367
recall =  0.1163101604278075
f1 score =  0.1766497461928934
